# Shear bias estimation

Martin Kilbinger, Arnau Pujol

This example notebook creates galaxy and PSF images (using galsim), measures their shapes
(with KSB/shapelens), computes the shear bias (shear response matrix), and creates a plot
of the bias as function of binned galaxy properties.

First, make sure the shear_bias package is installed and can be found.
Install the package as follows:
```bash
cd shear_bias
[sudo] python setup.py install [--prefix=<PATH>]
```
You might need super-user rights (when installing with `sudo`). Alternatively, you can specify a local path for <PATH>, e.g. ~/.local.

In [1]:
from shear_bias import *
%matplotlib inline

## Setup

Variables, file paths, and the job control are set below.

### Job control

#### Flow

In [2]:
job = param()

# Set re_run to True (False) if re-runs of jobs should overwrite (keep)
# previously produced output files
job.re_run = False

# Set dry_run to True (False) for jobs to run in test (real) mode
job.dry_run = False

#### Program choices

In [3]:
prog = param()

# Simulation generator, options: 'galsim'
prog.gen_sim = {}
prog.gen_sim['name'] = 'galsim'
prog.gen_sim['type'] = 'cmd'

# Shape measurement, options:
#   'get_shapes':  DEIMOS/shapelens, on command line, KSB or DEIMOS methd, type='cmd'
#   'galsim':      galsim using python interface, various methods
prog.shapes = {}
prog.shapes['name'] = 'get_shapes'
prog.shapes['type'] = 'cmd'

# Check whether required programs and/or libraries are installed
for pr in prog.get_vals():
    check_avail(pr)

### Shear values

In [4]:
# Small shear change for numerical derivative
dg = 0.02

In [5]:
# List of signs for shear change for the two shear components.

# Five steps (one in each direction + (0, 0)
g_steps = [(-1, 0), (0, -1), (1, 0), (0, 1), (0, 0)]

In [6]:
# Create g_dict, dictionary of shear values with step tuples as keys.
g_dict = {}
for step in g_steps:
    g_dict[step] = (step[0] * dg, step[1] * dg)
g_values = g_dict.values()

### Number of images

In [7]:
# Number of galaxy postage stamps per image is nxy_tiles^2.
# If this number is modified, all output files from a previous
# run should be deleted.
nxy_tiles = 10
#nxy_tiles = 100

# Number of files with different constant shear and PSF.
# This parameter can be changed without deleting previous output files.
nfiles = 5
#nfiles = 200

### File names and directories

#### Galsim

In [8]:
# Config files
galsim_config_fname = 'csc_multishear.yaml'
galsim_config_psf_fname = 'csc_psf.yaml'
galsim_config_dir = 'config/galsim'

# In- and outout directories
import os
great3_branch = 'control/space/constant'
galsim_input_dir = 'input/simulations/galsim/great3/{}'.format(great3_branch)
galsim_output_base_dir = 'output/galsim'

# Image file name formats
galsim_output_gal_fname_format = 'image-%03d-%1d.fits'
galsim_output_psf_fname_format = 'starfield_image-%03d-%1d.fits'

#### KSB-shapelens

In [9]:
# Set input directory (= galsim output directory) and output directory
ksb_input_base_dir  = galsim_output_base_dir
ksb_output_base_dir = 'output/shapelens'

#### Shear response

In [10]:
# Output directory for shear response files
R_output_dir = 'output/R'

## 2. Main program

### Download GREAT3 meta-data and PSF images.
Already downloaded files are skipped.

In [11]:
remote_dir = 'https://github.com/martinkilbinger/ede18_shear_cal/blob/master/data/great3/{}'\
   .format(great3_branch)
n_downloaded = download_great3_data(galsim_input_dir, remote_dir, great3_branch, nfiles, mode='?raw=true')

File input/simulations/galsim/great3/control/space/constant/epoch_catalog-000-0.fits exists, skipping...
14 files downloaded
*** End download_great3_data ***


### Create images with galsim

In [12]:
# Set paths
galsim_config_path     = '{}/{}'.format(galsim_config_dir, galsim_config_fname)
galsim_config_psf_path = '{}/{}'.format(galsim_config_dir, galsim_config_psf_fname)

# Call galsim
create_all_sims_great3(g_values, galsim_config_path, galsim_config_psf_path, \
        galsim_input_dir, galsim_output_base_dir, galsim_output_gal_fname_format, \
        galsim_output_psf_fname_format, nxy_tiles=nxy_tiles, nfiles=nfiles, job=job)

*** Start create_all_sims_great3 ***
For shear (0.0,0.02), 5 images need to be created, running 'galsim'
running galsim config/galsim/csc_multishear.yaml gal.shear.g1=0.0 gal.shear.g2=0.02 input.catalog.dir=input/simulations/galsim/great3/control/space/constant input.dict.dir=input/simulations/galsim/great3/control/space/constant output.dir=output/galsim/g1_0.0_g2_0.02 output.file_name.format=image-%03d-%1d.fits output.nfiles=5 image.nx_tiles=10 image.ny_tiles=10
Using config file config/galsim/csc_multishear.yaml
Using 4 processes for file processing
File may have been truncated: actual file length (1073152) is smaller than the expected size (3692160)
Process-1: Exception caught for file 0 = output/galsim/g1_0.0_g2_0.02/image-000-0.fits
Traceback (most recent call last):
File "/home/ede2019/miniconda3/envs/tr2/lib/python2.7/site-packages/galsim/config/process.py", line 851, in worker
result = job_func(**kwargs)
File "/home/ede2019/miniconda3/envs/tr2/lib/python2.7/site-packages/galsim

Start file 1 = output/galsim/g1_0.02_g2_0.0/image-001-0.fits
Start file 2 = output/galsim/g1_0.02_g2_0.0/image-002-0.fits
Start file 3 = output/galsim/g1_0.02_g2_0.0/image-003-0.fits
Start file 4 = output/galsim/g1_0.02_g2_0.0/image-004-0.fits
Process-3: File 3 = output/galsim/g1_0.02_g2_0.0/image-003-0.fits: time = 10.959217 sec
Process-1: File 4 = output/galsim/g1_0.02_g2_0.0/image-004-0.fits: time = 10.939846 sec
Process-4: File 2 = output/galsim/g1_0.02_g2_0.0/image-002-0.fits: time = 18.412728 sec
Process-2: File 1 = output/galsim/g1_0.02_g2_0.0/image-001-0.fits: time = 19.614415 sec
Total time for 4 files with 4 processes = 20.026837 sec
Done building files
For shear (0.0,0.0), 5 images need to be created, running 'galsim'
running galsim config/galsim/csc_multishear.yaml gal.shear.g1=0.0 gal.shear.g2=0.0 input.catalog.dir=input/simulations/galsim/great3/control/space/constant input.dict.dir=input/simulations/galsim/great3/control/space/constant output.dir=output/galsim/g1_0.0_g2_

### Measure shapes with KSB (shapelens)

In [ ]:
# call get_shapes
all_shapes_shapelens(g_values, ksb_input_base_dir, ksb_output_base_dir, nfiles, nxy_tiles, job=job)

*** Start all_shapes_shapelens ***
running get_shapes -T -g 10 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_0.02/image-000-0.fits > output/shapelens/g1_0.0_g2_0.02/result-000.txt
Last call returned error code 34304
running get_shapes -T -g 10 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_-0.02/image-000-0.fits > output/shapelens/g1_0.0_g2_-0.02/result-000.txt
Last call returned error code 34304
running get_shapes -T -g 10 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.02_g2_0.0/image-000-0.fits > output/shapelens/g1_0.02_g2_0.0/result-000.txt
Last call returned error code 34304
running get_shapes -T -g 10 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_0.0_g2_0.0/image-000-0.fits > output/shapelens/g1_0.0_g2_0.0/result-000.txt
Last call returned error code 34304
running get_shapes -T -g 10 -p output/galsim/psf/starfield_image-000-0.fits output/galsim/g1_-0.02_g2_0.0/image-000-0.fits > output/shape

### Read shear estimate results from files

In [ ]:
# Set input directories = KSB and galsim output directories
results_input_base_dir  = ksb_output_base_dir
psf_input_dir           = '{}/psf/'.format(galsim_output_base_dir)

# Read files
results = all_read_shapelens(g_dict, results_input_base_dir, psf_input_dir, nfiles, nobj_per_file_exp=nxy_tiles**2)

### Compute shear response matrix and save to file

In [ ]:
# Compute shear response
R = shear_response(results, dg, output_dir=R_output_dir)

### Compute and print mean and std of shear response matrix

In [ ]:
# If not compute above, shear response can be read from files
#R = read_R(R_output_dir)

print('mean(R)')
print(R.mean(axis=2))

print('std(R)')
print(R.std(axis=2))

### Plot shear response

In [ ]:
# Variable on x-axis
xvar  = results[(0, 0)].sn

# Variable name = xlabel
xname = 'SNR'

# Number of x-bins
nbins = 3

# Variables on y-axis, can be array for multiple curves in plot
yvar   = [shear_bias_m(R, i) for i in [0, 1]]
yvar.append(R[0,1])

# Variable names for legend
yname  = ['$m_1$', '$m_2$', '$R_{12}$']

# Color of points and error bars
color  = ['g', 'r', 'b']

# Point types
marker = ['o', 's', 'd']

# Create plot and save to file
x_mean, y_mean, y_std = \
    plot_mean_per_bin_several(xvar, xname, yvar, yname, nbins, error_mode='std', \
                              color=color, lw=2, marker=marker, out_name='snr_m1.pdf')